In [38]:
import numpy as np
import sklearn
from scipy.linalg import khatri_rao

# You are allowed to import any submodules of sklearn that learn linear models e.g. sklearn.svm etc
# You are not allowed to use other libraries such as keras, tensorflow etc
# You are not allowed to use any scipy routine other than khatri_rao

# DO NOT CHANGE THE NAME OF THE METHODS my_fit, my_map etc BELOW
# THESE WILL BE INVOKED BY THE EVALUATION SCRIPT. CHANGING THESE NAMES WILL CAUSE EVALUATION FAILURE

# You may define any new functions, variables, classes here
# For example, functions to calculate next coordinate or step length

################################
# Non Editable Region Starting #
################################
def my_fit( X_train, y_train ):
################################
#  Non Editable Region Ending  #
################################

    # Use this method to train your model using training CRPs
    # X_train has 32 columns containing the challeenge bits
    # y_train contains the responses

    # THE RETURNED MODEL SHOULD BE A SINGLE VECTOR AND A BIAS TERM
    # If you do not wish to use a bias term, set it to 0

    X_train_feature_map = my_map(X_train)

    #continue from here
    # Train a linear CSVM with C=1
    from sklearn.svm import LinearSVC
    svm_model = LinearSVC(C=1, max_iter=10000, dual=True)
    svm_model.fit(X_train_feature_map, y_train)

    # Extract weight vector and bias term
    w = svm_model.coef_.flatten()  # Convert weights to 1D array
    b = svm_model.intercept_[0]    # Extract bias



    return w,b

################################
# Non Editable Region Starting #
################################
def my_map( X ):
################################
#  Non Editable Region Ending  #
################################
    # Create polynomial features
    feat=np.zeros((X.shape[0],528),dtype='int')
    u=np.hstack((1-2*X,np.ones((X.shape[0],1),dtype='int')))
    for i in range(30,-1,-1):
      u[:,i]=np.multiply(u[:,i],u[:,i+1])
    for p in range(0,X.shape[0]):
    # # Use this method to create features.
    # It is likely that my_fit will internally call my_map to create features for train points
      v=u[p,:].reshape(33,1)
      result=np.triu(v*v.T,k=1)
      feat[p]=result[result!=0]
    return feat

In [39]:
Z_trn = np.loadtxt( "train.dat" )
Z_tst = np.loadtxt( "test.dat" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [40]:
import time as tm
for t in range( n_trials ):
	tic = tm.perf_counter()
	w,b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic
	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = np.dot(feat,w) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

In [41]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map, 1 - acc ,acc*100)

528.0 59.24134084979996 0.3426931776000856 0.008099999999999996 99.19
